In [5]:
import pandas as pd
#빌라주택, 원룸투룸  데이터 가져오기 
data = pd.read_csv('../preprocessing_data/최종/living_alone.csv')
data.head()
len(data)

98

In [98]:
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import time
from collections import defaultdict
from selenium import webdriver
import random

def circle_data_crawling(circle_id, circle_num):
    lst = []
    #피터팬은 걸러놔야해 

    try:
        #아이디가 없는것도 있다 그런거는 스킵
        driver.find_element_by_xpath(f'//*[@id="{circle_id}"]').send_keys(Keys.ENTER)
    except:
        return []
    #밑으로 내려주기 내려주는거 횟수 저장하기 
    for _ in range(((circle_num // 20 )) * 5):
        itemlist = driver.find_element_by_xpath('//*[@id="listContents1"]/div')
        driver.execute_script("arguments[0].scrollBy(0, document.body.scrollHeight)", itemlist)
        
        #일정한 시간에 클릭을 하니까 벤하는거 같다 
        time.sleep(0.5)
    
    time.sleep(0.5)
    piterpen = driver.page_source
    piterpen_soup = bs(piterpen, 'html.parser')
    piterpen_lst = []
    
    for num, string in enumerate(piterpen_soup.find_all('div', class_ = 'item_inner'), start = 1):
        if ('중개사' in string.find('div', class_= 'label_area').text):
            data = defaultdict(str)
            #피터팬, 중개소 2곳은 목록값만 저장
            data['제목'] = string.find('div', class_ = 'item_title').text
            data['가격'] = string.find('div', class_ = 'price_line').text
            data['소제목'] = string.find('div', class_ = 'info_area').text
            data['부제'] = string.find('div', class_ = 'tag_area').text
            # data['공인중개소'] = string.find('div', class_ = 'cp_area').text
            lst.append(data)
            piterpen_lst.append(num)
            continue

        if ('피터팬' in string.find('div', class_ = 'cp_area').find('a').text):

            data = defaultdict(str)
            #피터팬, 중개소 2곳은 목록값만 저장
            data['제목'] = string.find('div', class_ = 'item_title').text
            data['가격'] = string.find('div', class_ = 'price_line').text
            data['소제목'] = string.find('div', class_ = 'info_area').text
            data['부제'] = string.find('div', class_ = 'tag_area').text
            data['공인중개소'] = string.find('div', class_ = 'cp_area').text
            lst.append(data)
            piterpen_lst.append(num)
    

    for i in range(1, circle_num + 1):
        if i in piterpen_lst: continue
        data = defaultdict(str)
        # number = random.uniform(0, 3)
        # time.sleep(number)
        #제목누르기 사진이 있는거와 없는거가 찾이가 난다 
        try:
            driver.find_element_by_xpath(f'//*[@id="listContents1"]/div/div/div/div[{i}]/div/a/div[1]/span').click()
            #사진 없음
            data['사진'] = 0
        except:
            driver.find_element_by_xpath(f'//*[@id="listContents1"]/div/div/div/div[{i}]/div/a[2]/div[1]/span').click()
            data['사진'] = 1

        time.sleep(0.5)
        estimate = driver.page_source
        estimate_soup = bs(estimate, 'html.parser')
        #옆창이 안나오는 경우도 있다 중개소가 여러곳인곳
        try:
            detail_estimate = estimate_soup.find('div', class_ = 'detail_contents_inner')

            data['제목'] = detail_estimate.find('h4').text # 제목
            data['가격'] = detail_estimate.find('div', class_ ='info_article_price' ).text # 가격
            data['소제목'] = detail_estimate.find('div', class_ ='info_article_feature' ).text #소제목
            
            # 표 데이터 
            for i in detail_estimate.find('div', class_ = 'detail_box--summary').find_all('tr'):
                for title, content  in zip(i.find_all('th'), i.find_all('td')):
                    data[title.text] = content.text
        except: #중개사가 2곳
            #소 제목만 가져오자 
            

            detail_estimate = estimate_soup.find('div', class_ = 'detail_contents_inner')
            data['제목'] = detail_estimate.find('h4').text # 제목
            data['가격'] = detail_estimate.find('div', class_ ='info_article_price' ).text # 가격
            data['소제목'] = detail_estimate.find('div', class_ ='info_article_feature' ).text #소제목
            # 표 데이터 
            for i in detail_estimate.find('div', class_ = 'detail_box--summary').find_all('tr'):
                for title, content  in zip(i.find_all('th'), i.find_all('td')):
                    data[title.text] = content.text



        #데이터 저장
        lst.append(data)

    return lst

boodongsan_data = []
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument('--user-agent=' + user_agent)
driver = webdriver.Chrome('../chromedriver.exe', options= options)

for x, y in zip(data['x'], data['y']):
    driver.get(f'https://new.land.naver.com/rooms?ms={y},{x},19&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT&ad=true')
    time.sleep(3)
    page = driver.page_source
    soup = bs(page, 'html.parser')
    circle = soup.find('div',class_ = 'map_area').select('div > div > a')
    print(f'좌표 : {y},{x}, 원 갯수 {len(circle)}')
    for i in circle:
        # try:
            circle_id = i['id']
            circle_num = i.select_one('span.sale_number').text.split('개')[0]
            boodongsan_data.extend(circle_data_crawling(circle_id,int(circle_num)))
            print(len(boodongsan_data))
        # except:
        #     break
        

boodongsan_data = pd.DataFrame(boodongsan_data)

좌표 : 37.5213956191,126.8963096106, 원 갯수 21
1


AttributeError: 'NoneType' object has no attribute 'text'

In [23]:
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import time
from collections import defaultdict
from selenium import webdriver
import random

boodongsan_data = []
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument('--user-agent=' + user_agent)
driver = webdriver.Chrome('../chromedriver.exe', options= options)

driver.get(f'https://new.land.naver.com/rooms?ms=37.5247754,126.9034063,19&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT&ad=true')
driver.implicitly_wait(5)
time.sleep(0.5)
page = driver.page_source
soup = bs(page, 'html.parser')
circle = soup.find('div',class_ = 'map_area').select('div > div > a')
for i in circle:

            circle_id = i['id']
            circle_num = i.select_one('span.sale_number').text.split('개')[0]


KeyError: 'id'

In [119]:
estimate = driver.page_source
estimate_soup = bs(estimate, 'html.parser')
estimate_soup.find_all('div', class_= 'label_area')[6].text


'확인22.11.22.'

In [130]:
piterpen = driver.page_source
piterpen_soup = bs(piterpen, 'html.parser')
for i in range(10):
    print(piterpen_soup.find_all('div', class_ = 'item_inner')[i].find('div', class_= 'label_area').text)
    if '중개사' in piterpen_soup.find_all('div', class_ = 'item_inner')[i].find('div', class_= 'label_area').text:
        print('yes')
    else:
        print('no')

확인22.12.13.
no
확인22.12.20.
no
확인22.12.15.
no
확인22.12.14.
no
확인22.12.14.중개사2곳
yes


AttributeError: 'NoneType' object has no attribute 'text'